#### CSC 180 Intelligent Systems 

#### William Lorence

#### California State University, Sacramento


# Project 2: Modern Low Footprint Cyber Attack Protection
## Reading the Data
The code below reads the data from the dataset and creates dataframes. Values of "-" are treated as N/A and entries with this value are dropped from the dataframe.

In [79]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

path = "./dataset/"
save_path = "./models/"

#Defines filepaths for the data sets
training_set = os.path.join(path,"UNSW_NB15_training-set.csv")
test_set = os.path.join(path,"UNSW_NB15_test-set.csv")

#Loads files into dataframes
df_training_set = pd.read_csv(training_set, na_values = ['-'])
df_test_set = pd.read_csv(test_set, na_values = ['-'])

#Removes rows with a "-" in any column
df_training_set.dropna(inplace = True)
df_test_set.dropna(inplace = True)

print(len(df_training_set))
print(len(df_test_set))

df_training_set.head()

81173
35179


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
11,12,2.093085,tcp,smtp,FIN,62,28,56329,2212,42.520967,62,252,2.118251e+05,8152.559082,28,8,34.312868,75.092445,3253.278833,106.113453,255,1824722662,860716719,255,0.131140,0.052852,0.078288,909,79,0,0,2,1,1,1,1,2,0,0,0,1,1,0,Normal,0
15,16,0.000002,udp,snmp,INT,2,0,138,0,500000.001300,254,0,2.760000e+08,0.000000,0,0,0.002000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,69,0,0,0,1,2,1,1,1,4,0,0,0,2,1,0,Normal,0
17,18,0.393556,tcp,http,FIN,10,8,860,1096,43.195886,62,252,1.573347e+04,19494.048830,2,2,43.728444,47.669145,2124.837873,71.805867,255,3882971404,3084071099,255,0.120313,0.059201,0.061112,86,137,1,103,3,1,1,1,1,2,0,0,1,1,3,0,Normal,0
21,22,0.338017,tcp,http,FIN,10,6,998,268,44.376468,254,252,2.127704e+04,5301.508789,2,1,35.218111,56.579801,1928.550710,82.022258,255,2665974075,3521361798,255,0.154433,0.055109,0.099324,100,45,1,0,6,1,1,1,1,1,0,0,1,2,3,0,Normal,0


## Data Filtering
The code below removes categorical values that are not present in both datasets. As visible via the print statements, roughly 600 entries are dropped from the training set, while only one is dropped from the test set.

In [80]:
#Removes categorical values not present in both datasets
categorical_columns = ['proto', 'service', 'state', 'attack_cat']

for column in categorical_columns:
    unique_values_training = set(df_training_set[column].unique())
    unique_values_test = set(df_test_set[column].unique())

    print(unique_values_training)
    print(unique_values_test)
    
    common_values = unique_values_training.intersection(unique_values_test)
    print(common_values)

    df_training_set = df_training_set[df_training_set[column].isin(common_values)]
    df_test_set = df_test_set[df_test_set[column].isin(common_values)]

print(len(df_training_set))
print(len(df_test_set))

{'udp', 'tcp'}
{'udp', 'tcp'}
{'tcp', 'udp'}
{'smtp', 'irc', 'radius', 'ssl', 'ssh', 'dns', 'snmp', 'ftp-data', 'pop3', 'http', 'ftp', 'dhcp'}
{'smtp', 'irc', 'radius', 'ssl', 'ssh', 'dns', 'snmp', 'ftp-data', 'pop3', 'http', 'ftp', 'dhcp'}
{'smtp', 'irc', 'radius', 'ssl', 'ssh', 'dns', 'snmp', 'ftp-data', 'pop3', 'http', 'ftp', 'dhcp'}
{'CON', 'INT', 'RST', 'FIN', 'REQ'}
{'CON', 'REQ', 'INT', 'FIN', 'ACC'}
{'CON', 'INT', 'FIN', 'REQ'}
{'Backdoor', 'Fuzzers', 'Worms', 'Analysis', 'Generic', 'DoS', 'Exploits', 'Normal', 'Reconnaissance'}
{'Backdoor', 'Fuzzers', 'Worms', 'Generic', 'DoS', 'Exploits', 'Normal', 'Reconnaissance'}
{'Backdoor', 'Fuzzers', 'Worms', 'Generic', 'DoS', 'Exploits', 'Normal', 'Reconnaissance'}
80595
35178


The columns 'id' and 'is_sm_ips_ports' are dropped from the dataframes: 'id' is irrelevent to the data at hand, and 'is_sm_ips_ports' causes errors when calculating z scores (likely because it is always 0).

In [81]:
df_training_set.drop('id', axis = 1, inplace = True)
df_training_set.drop('is_sm_ips_ports', axis = 1, inplace = True)

df_test_set.drop('id', axis = 1, inplace = True)
df_test_set.drop('is_sm_ips_ports', axis = 1, inplace = True)

The numerical data is then normalized via z-score.

In [82]:
#Finds numerical data (columns that are not categorical)
numerical_columns = set(df_training_set.columns.symmetric_difference(categorical_columns))

from scipy.stats import zscore

def z_score_numerical(df, names):
    for name in names:
        df[name] = zscore(df[name])

z_score_numerical(df_training_set, numerical_columns)
z_score_numerical(df_test_set, numerical_columns)

Finally, the next bit of code encodes the now-filtered categorical values into their own columns.

In [83]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue) given a dataframe and a list of column names
def encode_text_dummy_loop(df, names):
    for name in names:
        dummies = pd.get_dummies(df[name])
        for x in dummies.columns:
            dummy_name = "{}-{}".format(name, x)
            df[dummy_name] = dummies[x]
        df.drop(name, axis=1, inplace=True)

encode_text_dummy_loop(df_training_set, categorical_columns)
encode_text_dummy_loop(df_test_set, categorical_columns)

The data is now ready for processing.